## BETTER WAY 23 - 인터페이스가 간단하면 클래스 대신 함수를 받자
## Item23 - Accept Functions for Simple Interfaces Instead of Classes

파이썬의 내장 API 상당수에는 함수를 넘겨서 동작을 사용자화 하는 기능이 있다. API는 이런 후크(hook)를 이용해서 우리가 작성한 코드를 실행 중에 호출한다. <br />
예를 들어 list 타입의 sort 메소드는 정렬에 필요한 각 인덱스의 값을 결정하는 선택적 key 인수를 받는다. 아래의 예제코드는 `lambda` 표현식을 `key` 후크로 넘겨서 `name` 이라는 리스트를 길이`len()`로 정렬한다.

In [1]:
names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']
names.sort(key=lambda x: len(x))
print(names)

['Plato', 'Socrates', 'Aristotle', 'Archimedes']


파이썬이 **일급함수(first-class function)**이기 때문에 함수가 후크로 동작할 수 있다. 일급함수에 대한 설명은 [BETTER WAY 15](https://github.com/ExcelsiorCJH/Study/blob/master/Python/Effective-python/Chap02-Functions/Item15-Know_How_Closures_Interact_with_Variable_Scope.ipynb)를 참고하면 된다. <br />
예를들어 `defaultdict` 클래스의 동작을 사용자화 한다고 해보자. defaultdict에 대한 설명은 [여기서](http://excelsior-cjh.tistory.com/entry/collections-%EB%AA%A8%EB%93%88-defaultdict?category=966334) 참고하자. 이 자료구조는 찾을 수 없는 키에 접근할 때마다 호출될 함수를 받는다. `defaultdict`에 넘길 함수는 딕셔너리에서 찾을 수 없는 키에 대응할 기본값(default)을 반환해야 한다. <br />
아래의 예제는 키를 찾을 수 없을 때마다 로그를 남기고 기본값으로 `0`을 반환하는 후크를 정의한 코드이다.

In [2]:
from collections import defaultdict

# 후크(hook) 정의
def log_missing():
    print('Key added')
    return 0

current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]
result = defaultdict(log_missing, current)
print('Befor:', dict(result))

for key, amount in increments:
    result[key] += amount

print('After:', dict(result))

Befor: {'blue': 3, 'green': 12}
Key added
Key added
After: {'orange': 9, 'red': 5, 'blue': 20, 'green': 12}


이번에는 기본값 후크를 defaultdict에 넘겨서 찾을 수 없는 키의 총 개수를 센다고 해보자. 다음은 클로저를 이용해 만든 헬퍼함수이다.

In [3]:
def increment_with_report(current, increments):
    added_count = 0
    
    def missing():
        nonlocal added_count  # 상태 보존 클로저
        added_count += 1
        return 0
    
    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount
        
    return result, added_count

In [4]:
result, count = increment_with_report(current, increments)
assert count == 2
print(count)

2


하지만 위의 문제는 함수를 이해하기가 어려운 문제가 있다. 또 다른 방법으로는 보존할 상태를 캡슐화하는 작은 클래스를 정의하는 즉, 헬퍼클래스를 사용하는 방법이 있다.

In [5]:
class CountMissing(object):
    def __init__(self):
        self.added = 0
        
    def missing(self):
        self.added += 1
        return 0

파이썬에서는 일급 함수 덕분에 객체로 `CountMissing.missing` 메서드를 직접 참조해서 defaultdict의 기본값 후크로 넘길 수 있다.

In [6]:
counter = CountMissing()
result = defaultdict(counter.missing, current)

for key, amount in increments:
    result[key] += amount

print(counter.added)

2


위의 클로저 함수보다는 이해하기가 쉽지만 CountMissing 클래스 자체만으로는 용도가 무엇인지 바로 이해하기 어렵다. <br />
파이썬에서는 클래스에 `__call__` 이라는 메소드를 정의해서 이런 상황을 명확하게 할 수 있다. `__call__` 메서드는 객체를 함수처럼 호출할 수 있게 해준다. 또한 내장 함수 `callable`이 이런 인스턴스에 대해서는 True를 반환하게 만든다.

In [7]:
class BetterCountMissing(object):
    def __init__(self):
        self.added = 0
        
    def __call__(self):
        self.added += 1
        return 0
    

counter = BetterCountMissing()
counter()
print(callable(counter))

True


다음은 BetterCountMissing 인스턴스를 defaultdict의 기본값 후크로 사용하여 딕셔너리에 없어서 새로 추가된 키의 개수를 알아내는 코드다.

In [8]:
counter = BetterCountMissing()
result = defaultdict(counter, current)  # __call__ 이 필요함
for key, amount in increments:
    result[key] += amount

print(counter.added)

2


`__call__` 메서드는 (API 후크처럼) 함수 인수를 사용하기 적합한 위치에 클래스의 인스턴스를 사용할 수 있다는 사실을 드러낸다.

### 정리
- 파이썬에서 컴포넌트 사이의 간단한 인터페이스용으로 클래스를 정의하고 인스턴스를 생성하는 대신에 함수만 써도 충분한 경우가 있다.
- `__call__` 이라는 메소드는 클래스의 인스턴스를 일반 파이썬 함수처럼 호출할 수 있게 해준다.
- 상태를 보존하는 함수가 필요할 때 상태 보존 클로저를 정의하는 대신 `__call__` 메소드를 제공하는 클래스를 정의하는 것이 좋다. 